## Classification using TensorFlow's Estimator API

### Goal
Predicting whether Income exceeds $50k based on the census data.

### Dataset
Dataset can be downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Census+Income)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('census_data.csv')
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df.income_bracket.value_counts()

 <=50K    24720
 >50K      7841
Name: income_bracket, dtype: int64

Convert the output column to label whose value is 1 if the income is above $50k and 0 otherwise.

In [4]:
def fix_label(label):
    if label == ' <=50K':
        return 0
    else:
        return 1


In [5]:
df['income_bracket'] = df['income_bracket'].apply(fix_label)

In [6]:
df['income_bracket'].unique()

array([0, 1], dtype=int64)

Using sklearn's `train_test_split` method split the data into Train and Test set in the ratio 80:20

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop('income_bracket', axis = 1)
Y = df['income_bracket']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
import tensorflow as tf

In [10]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


Convert the Categorical columns using Tensorflow's Feature column API. You can make use of `categorical_column_with_vocabulary_list` if you are aware of all the possible values the column has or you can make use of `categorical_column_with_hash_bucket` if you are unsure of the values

In [11]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=10)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=100)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=100)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=100)
gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size=100)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size=100)


For Numeric Column we can make use of `numeric_column` 

In [12]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

In [13]:
feature_columns = [age, workclass, education, education_num, marital_status, occupation, relationship, race, gender, capital_gain, capital_loss, hours_per_week, native_country]

Since our dataset is in pandas, we will make use of `pandas_input_fn` to create an input function to pass it to the `train` method of the Classifier

In [14]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train,batch_size=128, num_epochs=10, shuffle=False)

In [15]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=2, model_dir='./output')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000008635045C88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Train the Classifier for 10 epochs with a batch size of 128

In [16]:
model.train(input_fn=input_func)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-6105
INFO:tensorflow:Saving checkpoints for 6106 into ./output\model.ckpt.
INFO:tensorflow:loss = 54.9291, step = 6106
INFO:tensorflow:global_step/sec: 96.9737
INFO:tensorflow:loss = 51.631, step = 6206 (1.036 sec)
INFO:tensorflow:global_step/sec: 144.436
INFO:tensorflow:loss = 38.4335, step = 6306 (0.692 sec)
INFO:tensorflow:global_step/sec: 113.28
INFO:tensorflow:loss = 44.6804, step = 6406 (0.884 sec)
INFO:tensorflow:global_step/sec: 102.2
INFO:tensorflow:loss = 49.7656, step = 6506 (0.978 sec)
INFO:tensorflow:global_step/sec: 106.449
INFO:tensorflow:loss = 55.3606, step = 6606 (0.939 sec)
INFO:tensorflow:global_step/sec: 109.055
INFO:tensorflow:loss = 63.0493, step = 6706 (0.916 sec)
INFO:tensorflow:global_step/sec: 118.115
INFO:tensorflow:loss = 62.0577, step = 6806 (0.846 sec)
INFO:tensorflow:global_step/sec: 107.115
INFO:tensorflow:loss = 45.8664, step = 6906 (0.935 sec)
INF

Evaluate the classifier on Test set to determine the accuracy

In [17]:
eval_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, y=y_test, batch_size=128, shuffle=False)
result = model.evaluate(input_fn=eval_fn)

INFO:tensorflow:Starting evaluation at 2018-01-20-17:43:43
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-8140
INFO:tensorflow:Finished evaluation at 2018-01-20-17:43:45
INFO:tensorflow:Saving dict for global step 8140: accuracy = 0.836634, accuracy_baseline = 0.75879, auc = 0.888563, auc_precision_recall = 0.706515, average_loss = 0.429574, global_step = 8140, label/mean = 0.24121, loss = 54.8592, prediction/mean = 0.233026


In [18]:
for key in sorted(result):
  print('%s: %s' % (key, result[key]))

accuracy: 0.836634
accuracy_baseline: 0.75879
auc: 0.888563
auc_precision_recall: 0.706515
average_loss: 0.429574
global_step: 8140
label/mean: 0.24121
loss: 54.8592
prediction/mean: 0.233026


In [19]:
predict_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size=128, shuffle=False)

In [20]:
predict = model.predict(input_fn=predict_fn)

In [21]:
predictions = list(predict)

INFO:tensorflow:Restoring parameters from ./output\model.ckpt-8140


In [22]:
final_prediction = [pred['class_ids'][0] for pred in predictions]

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(y_test, final_prediction))

             precision    recall  f1-score   support

          0       0.87      0.92      0.90      4942
          1       0.70      0.57      0.63      1571

avg / total       0.83      0.84      0.83      6513

